In [1]:
import random
from ortools.linear_solver import pywraplp
from pyspark.sql import SparkSession
import pandas as pd

# Initialize Spark session
spark = SparkSession.builder.appName("EmployeeTaskAllocation").getOrCreate()

# Generate synthetic data for employees and tasks
num_employees = 30  # For debugging, using a smaller number of employees
num_tasks = 2  # Assuming 100 tasks
locations = ["Location1", "Location2", "Location3"]
skills = ["Skill1", "Skill2", "Skill3"]

# Create synthetic employees data
employees = []
for i in range(num_employees):
    employees.append({
        'id': i,
        'locations': random.sample(locations, random.randint(1, len(locations))),  # Random locations
        'skills': random.sample(skills, random.randint(1, len(skills))),  # Random skills
        'holidays': random.sample(range(1, 31), random.randint(0, 1))  # Random holidays in a month
    })

# Create synthetic tasks data
tasks = []
for i in range(num_tasks):
    tasks.append({
        'id': i,
        'location': random.choice(locations),  # Single location for each task
        'skills_required': random.sample(skills, random.randint(1, len(skills))),  # Random required skills
        'start_date': random.randint(1, 15),  # Random start date in the first half of the month
        'duration': random.randint(1, 2),  # Random duration between 1 and 5 days
        'working_hours_per_day': random.randint(1, 8)  # Random working hours per day between 1 and 8
    })

# Convert the synthetic data to Spark DataFrames
df_employees = spark.createDataFrame(employees)
df_tasks = spark.createDataFrame(tasks)

# Broadcast the tasks DataFrame
broadcast_tasks = spark.sparkContext.broadcast(df_tasks.collect())


# Define the allocation function
def allocate_tasks(employee_partition):
    from ortools.linear_solver import pywraplp
    employees = list(employee_partition)
    tasks = broadcast_tasks.value
    # Create the solver using SCIP backend
    # (other options include GLPK, CBC, etc.)
    solver = pywraplp.Solver.CreateSolver('SCIP')
    num_employees = len(employees)
    num_tasks = len(tasks)

    if num_employees == 0 or num_tasks == 0:
        return []

    # Decision variables: x[e, t] is 1 if employee 'e' is assigned to task 't', otherwise 0
    assignment_vars = {}
    for emp_id in range(num_employees):
        for task_id in range(num_tasks):
            assignment_vars[emp_id, task_id] = solver.BoolVar(f'x[{emp_id},{task_id}]')

    # Constraints: Each task should be assigned to exactly one employee
    for task_id in range(num_tasks):
        task = tasks[task_id]
        solver.Add(solver.Sum(
            assignment_vars[emp_id, task_id] for emp_id in range(num_employees)
            if task['location'] in employees[emp_id]['locations'] and
            all(skill in employees[emp_id]['skills'] for skill in task['skills_required']) and
            task['start_date'] not in employees[emp_id]['holidays']
        ) == 1)

    # Constraints: Each employee can only work on one task at a time and cannot work more than 8 hours a day
    for emp_id in range(num_employees):
        for day in range(1, 32):  # Assuming a month has 31 days
            overlapping_tasks = [
                task_id for task_id in range(num_tasks)
                if tasks[task_id]['start_date'] <= day < tasks[task_id]['start_date'] + tasks[task_id]['duration']
            ]
            solver.Add(solver.Sum(
                assignment_vars[emp_id, task_id] * tasks[task_id]['working_hours_per_day']
                for task_id in overlapping_tasks
            ) <= 8)

    # Constraints: Each employee cannot work more than 40 hours per week
    for emp_id in range(num_employees):
        for week in range(5):  # Assuming 4 weeks in a month
            weekly_tasks = [
                task_id for task_id in range(num_tasks)
                if tasks[task_id]['start_date'] // 7 == week or
                (tasks[task_id]['start_date'] + tasks[task_id]['duration']) // 7 == week
            ]
            solver.Add(solver.Sum(
                assignment_vars[emp_id, task_id] * tasks[task_id]['working_hours_per_day']
                for task_id in weekly_tasks
            ) <= 40)

    # Objective: Maximize the number of assigned tasks
    solver.Maximize(solver.Sum(
        assignment_vars[emp_id, task_id] for emp_id in range(num_employees) for task_id in range(num_tasks)
    ))

    # Solve the problem
    status = solver.Solve()

    # Check the results
    assignments = []
    if status == pywraplp.Solver.OPTIMAL:
        for emp_id in range(num_employees):
            for task_id in range(num_tasks):
                if assignment_vars[emp_id, task_id].solution_value() > 0:
                    assignments.append((employees[emp_id]['id'], tasks[task_id]['id'], tasks[task_id]['start_date'], tasks[task_id]['duration']))
    else:
        print("Solver did not find an optimal solution")

    return assignments

# Apply the allocation function across partitions
results = df_employees.rdd.mapPartitions(allocate_tasks).collect()

# Debug: Print the results
print("Task Assignments:", results)

# Format the results into a calendar
calendar = pd.DataFrame(index=range(num_employees), columns=range(1, 32))
calendar.index.name = 'Employee_ID'

# Populate the calendar
for assignment in results:
    emp_id, task_id, start_date, duration = assignment
    for day in range(start_date, start_date + duration):
        if day in calendar.columns:
            calendar.at[emp_id, day] = f'Task{task_id}'

# Fill NaN with empty string
calendar = calendar.fillna('')

# Display the calendar
import IPython.display as display
display.display(calendar)

# Stop the Spark session
spark.stop()


ModuleNotFoundError: No module named 'ortools'

In [6]:
import random
import pandas as pd
from ortools.linear_solver import pywraplp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable

# Initialize Spark session
spark = SparkSession.builder \
    .appName("EmployeeTaskAllocation") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Generate synthetic data for employees and tasks
num_employees = 30
num_tasks = 100
locations = ["Location1", "Location2", "Location3"]
skills = ["Skill1", "Skill2", "Skill3"]

# Create synthetic employees data
employees = []
for i in range(num_employees):
    employees.append({
        'id': i,
        'locations': random.sample(locations, random.randint(1, len(locations))),  # Random locations
        'skills': random.sample(skills, random.randint(1, len(skills))),  # Random skills
        'holidays': random.sample(range(1, 31), random.randint(1, 5))  # Random holidays in a month
    })

# Create synthetic tasks data
tasks = []
for i in range(num_tasks):
    tasks.append({
        'id': i,
        'location': random.choice(locations),  # Single location for each task
        'skills_required': random.sample(skills, random.randint(1, len(skills))),  # Random required skills
        'start_date': random.randint(1, 15),  # Random start date in the first half of the month
        'duration': random.randint(1, 5),  # Random duration between 1 and 5 days
        'working_hours_per_day': random.randint(1, 8)  # Random working hours per day between 1 and 8
    })

# Convert the synthetic data to Spark DataFrames
df_employees = spark.createDataFrame(employees)
df_tasks = spark.createDataFrame(tasks)

# Save tasks to Delta Lake to track assignment status
df_tasks.withColumn("status", col("id") % 10) \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("/tmp/delta/tasks")

# Function to solve the task allocation problem for a partition of tasks
def allocate_tasks(partition, employee_list):
    num_employees = len(employee_list)
    task_list = list(partition)
    num_tasks = len(task_list)
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    assignment_vars = {}

    for emp_id in range(num_employees):
        for task_id in range(num_tasks):
            assignment_vars[emp_id, task_id] = solver.BoolVar(f'x[{emp_id},{task_id}]')

    for task_idx, task in enumerate(task_list):
        task_id = task['id']
        solver.Add(solver.Sum(
            assignment_vars[emp_id, task_idx] for emp_id in range(num_employees)
            if task['location'] in employee_list[emp_id]['locations'] and
            all(skill in employee_list[emp_id]['skills'] for skill in task['skills_required']) and
            task['start_date'] not in employee_list[emp_id]['holidays']
        ) == 1)

    for emp_id in range(num_employees):
        for day in range(1, 32):
            overlapping_tasks = [
                task_idx for task_idx, task in enumerate(task_list)
                if task['start_date'] <= day < task['start_date'] + task['duration']
            ]
            solver.Add(solver.Sum(
                assignment_vars[emp_id, task_idx] * task['working_hours_per_day']
                for task_idx in overlapping_tasks
            ) <= 8)

    for emp_id in range(num_employees):
        for week in range(5):
            weekly_tasks = [
                task_idx for task_idx, task in enumerate(task_list)
                if task['start_date'] // 7 == week or
                (task['start_date'] + task['duration']) // 7 == week
            ]
            solver.Add(solver.Sum(
                assignment_vars[emp_id, task_idx] * task['working_hours_per_day']
                for task_idx in weekly_tasks
            ) <= 40)

    solver.Maximize(solver.Sum(
        assignment_vars[emp_id, task_idx] for emp_id in range(num_employees) for task_idx in range(num_tasks)
    ))

    status = solver.Solve()

    assignments = []
    if status == pywraplp.Solver.OPTIMAL:
        for emp_id in range(num_employees):
            for task_idx, task in enumerate(task_list):
                if assignment_vars[emp_id, task_idx].solution_value() > 0:
                    assignments.append((employee_list[emp_id]['id'], task['id'], task['start_date'], task['duration']))
    return assignments

# Broadcast employee data to all worker nodes
employee_list = df_employees.collect()
employee_broadcast = spark.sparkContext.broadcast(employee_list)

# Read tasks from Delta Lake and distribute the processing using PySpark
delta_tasks = DeltaTable.forPath(spark, "/tmp/delta/tasks")

def process_partition(partition_id):
    # Fetch the partition of tasks for this partition ID
    task_subset = delta_tasks.toDF().filter(col("status") == partition_id).collect()
    if not task_subset:
        return []
    
    # Solve the allocation problem for this partition
    assignments = allocate_tasks(task_subset, employee_broadcast.value)
    
    # Update Delta table to mark assigned tasks
    assigned_task_ids = [task[1] for task in assignments]
    delta_tasks.update(
        condition=col("id").isin(assigned_task_ids),
        set={"status": "-1"}  # Mark as assigned
    )
    
    return assignments

# Parallel processing using partition IDs
all_assignments = spark.sparkContext.parallelize(range(10), 10).flatMap(process_partition).collect()

# Format the results into a calendar
calendar = pd.DataFrame(index=range(num_employees), columns=range(1, 32))
calendar.index.name = 'Employee_ID'

# Populate the calendar
for assignment in all_assignments:
    emp_id, task_id, start_date, duration = assignment
    for day in range(start_date, start_date + duration):
        if day in calendar.columns:
            calendar.at[emp_id, day] = f'Task{task_id}'

# Fill NaN with empty string
calendar = calendar.fillna('')

# Display the calendar
import IPython.display as display
display.display(calendar)

# Stop the Spark session
spark.stop()


Traceback (most recent call last):                                              
  File "/opt/spark/python/pyspark/serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
  File "/opt/spark/python/pyspark/context.py", line 466, in __getnewargs__
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.